In [1]:
from tensorflow import keras
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255.0,
                                                       vertical_flip = True,
                                                       horizontal_flip = True)

In [2]:
from keras.applications import DenseNet121

weights_path = r'C:\Users\vidha\Health Analytics\brucechou1983_CheXNet_Keras_0.3.0_weights.h5\brucechou1983_CheXNet_Keras_0.3.0_weights.h5'

conv_base = DenseNet121(weights = None,
include_top = False,
input_shape = (224, 224, 3))

for layer in conv_base.layers:
  layer.trainable = False

predictions = keras.layers.Dense(14, activation='sigmoid', name='predictions')(conv_base.output)

conv_base = keras.Model(inputs = conv_base.input, outputs = predictions)

conv_base.load_weights(weights_path)
        
conv_base.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

In [3]:
trainGen = datagen.flow_from_directory(
    r'C:\Users\vidha\Health Analytics\archive\Covid19-dataset\train',
    target_size = (224,224),
    batch_size = 32,
    shuffle = True,
    class_mode = 'categorical'
)

datatestgen = keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255.0)
                                                       
testGen = datatestgen.flow_from_directory(
    r'C:\Users\vidha\Health Analytics\archive\Covid19-dataset\test',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [4]:
x = conv_base.layers[-2].output
x = keras.layers.GlobalAveragePooling2D()(conv_base.output)
predictions = keras.layers.Dense(3, activation='softmax')(x)
model = keras.models.Model(inputs = conv_base.input, outputs = predictions)

In [5]:
for layer in conv_base.layers:
    layer.trainable = False

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.fit(trainGen, epochs=10, batch_size=32, validation_data = testGen)

Epoch 1/10
8/8 [==============================] - 50s 5s/step - loss: 1.0956 - accuracy: 0.4422 - val_loss: 1.0954 - val_accuracy: 0.3939
Epoch 2/10
8/8 [==============================] - 29s 4s/step - loss: 1.0936 - accuracy: 0.4422 - val_loss: 1.0948 - val_accuracy: 0.3939
Epoch 3/10
8/8 [==============================] - 28s 3s/step - loss: 1.0930 - accuracy: 0.4422 - val_loss: 1.0943 - val_accuracy: 0.3939
Epoch 4/10
8/8 [==============================] - 26s 3s/step - loss: 1.0919 - accuracy: 0.4422 - val_loss: 1.0939 - val_accuracy: 0.3939
Epoch 5/10
8/8 [==============================] - 27s 3s/step - loss: 1.0905 - accuracy: 0.4422 - val_loss: 1.0934 - val_accuracy: 0.3939
Epoch 6/10
8/8 [==============================] - 28s 4s/step - loss: 1.0900 - accuracy: 0.4422 - val_loss: 1.0930 - val_accuracy: 0.3939
Epoch 7/10
8/8 [==============================] - 28s 4s/step - loss: 1.0881 - accuracy: 0.4422 - val_loss: 1.0926 - val_accuracy: 0.3939
Epoch 8/10
8/8 [==================

In [8]:
model.save('CheXNetFinalProjectFCL.model')

INFO:tensorflow:Assets written to: CheXNetFinalProjectFCL.model\assets


INFO:tensorflow:Assets written to: CheXNetFinalProjectFCL.model\assets


In [9]:
model.evaluate(testGen, batch_size = 32)

3/3 [==============================] - 6s 1s/step - loss: 1.0915 - accuracy: 0.3939


[1.0914692878723145, 0.39393940567970276]

In [10]:
model.evaluate(trainGen)

8/8 [==============================] - 28s 3s/step - loss: 1.0862 - accuracy: 0.4422


[1.0862197875976562, 0.44223108887672424]